In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import keras_tuner as kt
import json

# 📁 Chemin des datasets Parquet
DATA_DIR = "processed"
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

def create_sequences(df, target_col, lookback, features):
    data = df[features]
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data.iloc[i - lookback:i].values)
        y.append(data.iloc[i][target_col])
    return np.array(X), np.array(y)


In [8]:
def build_simple_lstm(hp, input_shape):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hp.Int("units", 32, 128, step=32),
                                   input_shape=input_shape))
    model.add(tf.keras.layers.Dropout(hp.Float("dropout", 0.0, 0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float("lr", 1e-4, 1e-2, sampling="log")),
                  loss="mse")
    return model

def build_bidirectional_lstm(hp, input_shape):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(units=hp.Int("units", 32, 128, step=32)),
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Dropout(hp.Float("dropout", 0.0, 0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float("lr", 1e-4, 1e-2, sampling="log")),
                  loss="mse")
    return model

def build_cnn_lstm(hp, input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=hp.Int("filters", 16, 64, step=16),
                               kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.LSTM(units=hp.Int("units", 32, 128, step=32)),
        tf.keras.layers.Dropout(hp.Float("dropout", 0.0, 0.5, step=0.1)),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float("lr", 1e-4, 1e-2, sampling="log")),
                  loss="mse")
    return model

def build_stacked_lstm(hp, input_shape):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hp.Int("units1", 32, 128, step=32), return_sequences=True, input_shape=input_shape))
    model.add(tf.keras.layers.LSTM(units=hp.Int("units2", 32, 128, step=32)))
    model.add(tf.keras.layers.Dropout(hp.Float("dropout", 0.0, 0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float("lr", 1e-4, 1e-2, sampling="log")),
                  loss="mse")
    return model


In [8]:
import ast
from glob import glob
from tqdm import tqdm

# === CONFIG ===
TARGET_COL = 'demand'
MAX_TRIALS = 10
EPOCHS = 20
BATCH_SIZE = 32
VAL_SPLIT = 0.2

def get_features_for_file(zone, horizon):
    df1 = pd.read_csv(f'submission/features_selected_{horizon}.csv')
    
    df1['features'] = df1['features'].apply(ast.literal_eval)
    
    features = df1.loc[df1['zone'] == zone, 'features'].values[0]
    return [col for col in features if col != TARGET_COL]

# === ARCHITECTURES DISPONIBLES ===
model_builders = {
    "LSTM": build_simple_lstm,
    "BiLSTM": build_bidirectional_lstm,
    "CNN_LSTM": build_cnn_lstm,
    "Stacked_LSTM": build_stacked_lstm
}

# === BOUCLE SUR TOUS LES FICHIERS PARQUET ===
for filepath in tqdm(glob(f"{DATA_DIR}/*.parquet")):
    filename = os.path.basename(filepath).replace(".parquet", "")
    try:
        filename_base = filename.replace(".parquet", "")
        zone = filename_base.rsplit("_processed_", 1)[0]
        horizon = filename_base.rsplit("_processed_", 1)[1]
        lookback = 24 if horizon == "hourly" else 7
    except:
        print(f"Format invalide pour le fichier : {filename}")
        continue

    print(f"\nProcessing {zone} ({horizon})")

    df = pd.read_parquet(filepath).dropna()
    features = get_features_for_file(zone, horizon)

    # Assure que target est dans le DataFrame
    if TARGET_COL not in df.columns:
        print(f"Colonne '{TARGET_COL}' absente de {filename}")
        continue

    df = df[[*features, TARGET_COL]].dropna()
    X, y = create_sequences(df, TARGET_COL, lookback, features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    best_rmse = float('inf')
    best_model_name = ""
    best_model_instance = None
    best_metrics = {}

    for model_name, builder in model_builders.items():
        print(f"🔍 Tuning {model_name}...")

        tuner = kt.RandomSearch(
            lambda hp: builder(hp, input_shape=X_train.shape[1:]),
            objective="val_loss",
            max_trials=MAX_TRIALS,
            overwrite=True,
            directory="kt_tuning",
            project_name=f"{zone}_{horizon}_{model_name}"
        )

        tuner.search(X_train, y_train, validation_split=VAL_SPLIT,
                     epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

        best_model = tuner.get_best_models(1)[0]
        y_pred = best_model.predict(X_test).flatten()

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)

        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = model_name
            best_model_instance = best_model
            best_metrics = {"MAE": mae, "RMSE": rmse, "R2": r2}

    # === SAUVEGARDE ===
    model_filename = f"{zone}_{horizon}_{best_model_name}.h5"
    metrics_filename = f"{zone}_{horizon}_metrics.json"
    best_model_instance.save(os.path.join(MODEL_DIR, model_filename))

    with open(os.path.join(MODEL_DIR, metrics_filename), "w") as f:
        json.dump(best_metrics, f, indent=2)

    print(f"Meilleur modèle pour {zone}_{horizon} : {best_model_name}")
    print(json.dumps(best_metrics, indent=2))


  0%|          | 0/25 [00:00<?, ?it/s]

Format invalide pour le fichier : Baleares_processed

Processing Baleares (daily)
🔍 Tuning LSTM...


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step


...

C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


🔍 Tuning CNN_LSTM...


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step
🔍 Tuning Stacked_LSTM...


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Exception ignored in: <function AtomicFunction.__del__ at 0x00000250C2B12B60>
Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py", line 303, in __del__
    RUNTIME_FUNCTION_REFS.pop(key)
KeyboardInterrupt: 
Traceback (most recent call last):
  File "C:\Users\pc\AppDat

In [ ]:
import os
import json
import ast
from glob import glob
from tqdm import tqdm
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# === CONFIG ===
TARGET_COL = 'demand'
EPOCHS = 20
BATCH_SIZE = 32
VAL_SPLIT = 0.2
DATA_DIR = "processed"
MODEL_DIR = "models"   # à adapter

# === Modèles simplifiés sans Keras Tuner ===
def build_simple_lstm_fixed(input_shape):
    units = 64
    dropout = 0.2
    lr = 1e-3

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units=units, input_shape=input_shape),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="mse")
    return model

def build_bidirectional_lstm_fixed(input_shape):
    units = 64
    dropout = 0.2
    lr = 1e-3

    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=units), input_shape=input_shape),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="mse")
    return model

def build_cnn_lstm_fixed(input_shape):
    filters = 32
    units = 64
    dropout = 0.2
    lr = 1e-3

    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.LSTM(units=units),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="mse")
    return model

def build_stacked_lstm_fixed(input_shape):
    units1 = 64
    units2 = 64
    dropout = 0.2
    lr = 1e-3

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units=units1, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.LSTM(units=units2),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="mse")
    return model

# === Fonction pour récupérer features (identique à la tienne) ===
def get_features_for_file(zone, horizon):
    df1 = pd.read_csv(f'submission/features_selected_{horizon}.csv')
    df1['features'] = df1['features'].apply(ast.literal_eval)
    features = df1.loc[df1['zone'] == zone, 'features'].values[0]
    return [col for col in features if col != TARGET_COL]

# === Fonction pour créer des séquences (à définir selon ton code) ===
def create_sequences(df, target_col, lookback, feature_cols):
    # Attention : adapter selon ta fonction existante !
    X, y = [], []
    data = df[feature_cols + [target_col]].values
    for i in range(len(df) - lookback):
        X.append(data[i:i+lookback, :-1])
        y.append(data[i+lookback, -1])
    return np.array(X), np.array(y)

# === Dictionnaire des constructeurs fixes ===
model_builders = {
    "LSTM": build_simple_lstm_fixed,
    "BiLSTM": build_bidirectional_lstm_fixed,
    "CNN_LSTM": build_cnn_lstm_fixed,
    "Stacked_LSTM": build_stacked_lstm_fixed
}

# === Boucle principale ===
for filepath in tqdm(glob(f"{DATA_DIR}/*.parquet")):
    filename = os.path.basename(filepath).replace(".parquet", "")
    try:
        zone, horizon = filename.rsplit("_processed_", 1)
        lookback = 24 if horizon == "hourly" else 7
    except:
        print(f"Format invalide pour le fichier : {filename}")
        continue

    print(f"\nProcessing {zone} ({horizon})")

    df = pd.read_parquet(filepath).dropna()
    features = get_features_for_file(zone, horizon)

    if TARGET_COL not in df.columns:
        print(f"Colonne '{TARGET_COL}' absente de {filename}")
        continue

    df = df[[*features, TARGET_COL]].dropna()
    X, y = create_sequences(df, TARGET_COL, lookback, features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    best_rmse = float('inf')
    best_model_name = None
    best_model_instance = None
    best_metrics = None

    for model_name, builder in model_builders.items():
        print(f"Training {model_name}...")

        model = builder(input_shape=X_train.shape[1:])
        history = model.fit(
            X_train, y_train,
            validation_split=VAL_SPLIT,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=1
        )

        y_pred = model.predict(X_test).flatten()

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)

        print(f"{model_name} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = model_name
            best_model_instance = model
            best_metrics = {"MAE": mae, "RMSE": rmse, "R2": r2}

    # Sauvegarde du meilleur modèle et métriques
    model_filename = f"{zone}_{horizon}_{best_model_name}.h5"
    metrics_filename = f"{zone}_{horizon}_metrics.json"

    best_model_instance.save(os.path.join(MODEL_DIR, model_filename))
    with open(os.path.join(MODEL_DIR, metrics_filename), "w") as f:
        json.dump(best_metrics, f, indent=2)

    print(f"Best model for {zone} ({horizon}): {best_model_name}")
    print(json.dumps(best_metrics, indent=2))


ERROR! Session/line number was not unique in database. History logging moved to new session 196


  0%|          | 0/25 [00:00<?, ?it/s]

Format invalide pour le fichier : Baleares_processed

Processing Baleares (daily)
Training LSTM...


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 64ms/step - loss: 257873968.0000 - val_loss: 236116256.0000
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 256816544.0000 - val_loss: 236067952.0000
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 254773808.0000 - val_loss: 236019488.0000
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 255377168.0000 - val_loss: 235977072.0000
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 255427072.0000 - val_loss: 235935904.0000
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 257276560.0000 - val_loss: 235891568.0000
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 255698160.0000 - val_loss: 235841792.0000
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 254633264.0000 - val_loss: 235797152.0000
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 252946624.0000 - val_loss: 235753184.0000
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 251722656.0000 - val_loss: 235

...

LSTM -> MAE: 170.5137, RMSE: 229.6442, R2: 0.0384
Training BiLSTM...
Epoch 1/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 42s 18ms/step - loss: 396956.2812 - val_loss: 459747.0938
Epoch 2/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - loss: 334247.6562 - val_loss: 393178.9062
Epoch 3/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - loss: 277983.7188 - val_loss: 335419.7188
Epoch 4/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 230005.7656 - val_loss: 285717.5000
Epoch 5/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 189435.8594 - val_loss: 238179.5469
Epoch 6/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - loss: 153992.3750 - val_loss: 199520.6250
Epoch 7/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - loss: 123510.7578 - val_loss: 166474.2656
Epoch 8/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - loss: 98984.3828 - val_loss: 138539.5469
Epoch 9/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - loss: 79453.7578 - val_loss: 114759.6406
Epoch 10/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/s

C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 404140.8125 - val_loss: 488114.5938
Epoch 2/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 366461.3750 - val_loss: 447930.3750
Epoch 3/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - loss: 329706.0938 - val_loss: 410042.2500
Epoch 4/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 298646.8438 - val_loss: 374852.1875
Epoch 5/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 267808.7812 - val_loss: 341311.0938
Epoch 6/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 239659.6562 - val_loss: 310348.2500
Epoch 7/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 213448.7969 - val_loss: 281435.0312
Epoch 8/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - loss: 192188.4531 - val_loss: 254161.9531
Epoch 9/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 169539.3906 - val_loss: 229036.8750
Epoch 10/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 150027.4688 - val_loss: 207114.4688
Epoch 11/20
877/877 ━━━━

C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 25s 21ms/step - loss: 388691.2812 - val_loss: 448146.1250
Epoch 2/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - loss: 324472.5312 - val_loss: 383786.9688
Epoch 3/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - loss: 270925.8438 - val_loss: 327022.4062
Epoch 4/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - loss: 224159.0625 - val_loss: 276833.6562
Epoch 5/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - loss: 184459.2500 - val_loss: 232588.6719
Epoch 6/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - loss: 148431.3281 - val_loss: 193975.0469
Epoch 7/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 118820.4141 - val_loss: 160817.7188
Epoch 8/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 23s 27ms/step - loss: 95205.1094 - val_loss: 132728.7188
Epoch 9/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - loss: 75745.0625 - val_loss: 109635.7344
Epoch 10/20
877/877 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - loss: 59160.5469 - val_loss: 91217.7812
Epoch 11/20
876/877 ━━━